# 7. Pattern matching in dependency trees

Today we will practice the ways in which dependency parsing can be used to extract complex syntactic structures from texts. SpaCy's *pattern matching* engine allows us to separate the description of the structure of interest from the code that finds it. The latter is mostly done by spaCy.

We will use a new dataset consisting of YLE English news from the recent weeks.

In [2]:
import csv
import spacy
import spacy.matcher

In [3]:
def read_articles(filename):
    result = []
    with open(filename) as fp:
        reader = csv.DictReader(fp)
        for row in reader:
            result.append(row['text'])
    return result

In [4]:
nlp = spacy.load('en_core_web_sm')
articles = read_articles('yle_en.csv')      # TODO supply the file path if needed
docs = [nlp(a) for a in articles]

In [5]:
docs[30][:50]

The construction of a new passenger ferry terminal in Kotka which will enable service to St. Petersburg, Russia is expected to begin this spring.
The terminal's final design is still in progress, but building work is expected to start as early as May this year

# In-class exercises

# Ex 1

Write a function `verb_dep(nlp, docs, tag, dep)`, that finds all **verbs** which take a dependent of type `dep` tagged as `tag`. The function should return a list of lists of pairs of `Token` objects: the verb and its dependent. We return a list of such pairs for every article (see example).

Use the function to find all instances of verbs that have a proper noun (`PROPN`) as subject (`nsubj`).

Use `spacy.matcher.DependencyMatcher` for implementation. You will find the necessary documentation here:
* https://spacy.io/api/dependencymatcher
* https://spacy.io/api/matcher

In [9]:
def verb_dep(nlp, docs, tag, dep):
    # find words tagged with 'tag' whose head and whose dependency type is 'dep'
    results = []
    for d in docs:
        results_docs = []
        for tok in d:
            if tok.pos_ == tag and tok.head.pos_ == 'VERB' and tok.dep_ == 'nsubj':
                results_docs.append((tok.head, tok))
        results.append(results_docs)
    return results

In [ ]:
def verb_dep(nlp, docs, tag, dep):
    pattern = [
        {'RIGHT_ID': 'verb',
         'RIGHT_ATTRS': {'POS': 'VERB'}}
        # verb -> subject
        {#TO-DO}
    ]

In [10]:
verb_dep(nlp, docs, 'PROPN', 'nsubj')[:2]

[[(announced, Nokia), (employs, Nokia), (said, Lundmark), (reported, Nokia)],
 [(said, Turkia), (weighed, Yle), (pick, Schoolchildren), (told, Kalo)]]

## Ex 2

Use `spacy.matcher.PhraseMatcher` to find the following multi-word phrases in the corpus:
* `"municipal elections"`
* `"coronavirus restrictions"`

In [236]:
# TODO create the `matcher` object


In [250]:
for i, d in enumerate(docs):
    for m in matcher(d):
        print(d[m[1]].sent)

Katri Mannonen / Yle
Municipalities scrambled to provide food to remote learning school children as most parts of the country entered a three-week period of heightened coronavirus restrictions, starting 8 March.
In addition, it was decided to postpone the municipal elections due to the difficult public health situation.
Rallies against coronavirus restrictions are taking place in some 40 countries around the world on Saturday.
Jussi Halla-aho, chair of the Finns Party, and the party's parliamentary group leader, Ville Tavio,on Saturday claimed Justice Minister Anna-Maja Henriksson had misled citizens and instructed municipalities to break the law in the lead-up to the decision to postpone this spring's municipal elections.
On Friday, Parliament's Constitutional Law Committee unanimously approved the postponement of the municipal elections from 18 April until 13 June.
According to senior police, Saturday's demonstration against coronavirus restrictions in downtown Helsinki will not set 

## Ex 3

Extract all syntactic phrases headed by the word `"situation"`.

In a dependency tree, such phrase corresponds to the subtree containing all descendents of the token of interest. You can get it using the field `token.subtree`.

In [6]:
for d in docs:
    for tok in d:
        if tok.lemma_ == 'situation':
            print(list(tok.subtree))

[the, situation]
[certain, situations, we, have, to, go, to, so, -, called, virgin, areas]
[a, similar, situation]
[the, prevailing, pandemic, situation, and, the, necessary, safety, arrangements]
[the, examination, situation]
[the, disease, situation]
[the, situation, in, Estonia, ,, where, the, infection, rate, is, now, the, second, -, highest, in, the, continent, and, around, nine, times, higher, than, it, is, in, Finland]
[the, worst, infection, situation, outside, of, the, Helsinki, region]
[The, coronavirus, situation]
[the, pandemic, situation]
[the, situation, in, the, greater, Helsinki, and, Turku, regions]
[Finland, 's, deteriorating, Covid, situation]
[the, situation]
[the, coronavirus, situation]
[an, embarrassing, situation]
[a, completely, everyday, situation]
[certain, situations]
[the, current, situation]
[situations, where, a, passenger, does, not, provide, proof, of, illness, or, a, recent, coronavirus, test]
[the, situation]
[The, situation]
[the, situation]
[the, de

# Homework

## Ex 4 (5p.)

Write a function `find_quotes(nlp, docs)` that identifies and attributes direct quotes in the text. A direct quote looks like this:

"*There is no simple answer,*" **Matti Sarvimäki**, Assistant Professor of Economics at Aalto University **told** the paper.

We recognize the following three identifying elements:
* **proposition** - the text that is being quoted (e.g. "There is no simple answer"),
* **author** - the person that is being quoted ("Matti Sarvimäki"),
* **cue** - a verb indicating a speech act, e.g. "told".

The function should output a list of triplets: `(author, cue, proposition)`, each being a `Tok` or `Span` object. You may use information gained from exercise 1 to manually create a list of speech act verbs (it does not need to be exhaustive, a couple of common examples are enough).

3 points are given for a basic working version of the function. For full points, pay attention to the following details:
* If a name is given in the form `FirstName LastName`, output a `Span` object containing them both as author. (1p.)
* If the proposition ends with a quote character (`"`), look for the matching character to find where it starts. It might be a different sentence! (1p.)

You can restrict the solution to propositions enclosed in quotes and the speaker being named by name (rather than a pronoun).

In [253]:
q = find_quotes(nlp, docs)

In [254]:
# This might not show *all* quotes. My solution is not perfect either.
print(q[3])

[(told, Hanna Nohynek, "Thinking about it, it could probably be a way to increase the number of people protected, if the quantity of vaccines available in Finland is much less than promised,), (said, Puhakka, "Oulu is a growing city and in certain situations we have to go to so-called virgin areas), (told, Puhakka, "We believe that Oulu's forest nature conservation decisions compensate for the decisions that have to be made in different parts of the city, due to the city's natural growth)]


In [255]:
print(q[207])

[(said, Simola, "My husband immediately said that it was a wolf), (told, Simola, "It was a really touching, amazing moment. Seeing such a wild animal inside the Ring I beltway really made you rub your eyes. It was a rare situation that doesn't happen every day), (said, Simola, "We've talked today about our relationship with wildlife and how even though the wolf may look nice, it is still an unpredictable wild animal that needs to be given its own space), (said, Hallamaa, "The wolf has not growled or shown its teeth. On the contrary, it has apparently tried to escape from humans with its tail between its legs)]


# Further reading

* https://spacy.io/usage/rule-based-matching
* https://applied-language-technology.readthedocs.io/en/latest/notebooks/part_iii/02_pattern_matching.html